<a href="https://colab.research.google.com/github/mateuszandzelak01/football_web_scraping/blob/main/football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import Libraries**

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

## **Test requests and BeautifulSoup Libraries**

In [3]:
url = 'https://www.transfermarkt.pl/pko-ekstraklasa/startseite/wettbewerb/PL1/plus/?saison_id=2013'
headers = {"User-Agent":"Mozilla/5.0"}
response = requests.get(url, headers=headers, verify=False)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', {'class' : 'items'})
row = table.findAll('tr')

## **Lech Poznań Example**

In [4]:
row[2].text

'\nLegia Warszawa Legia Warszawa4123,413587 tys. €24,05 mln €587 tys. €24,05 mln €'

## **Read the whole table with a loop**

In [5]:
team = []
squad = []
age = []
foreigners = []
total_market_value = []
average_market_value = []
for row in table.findAll('tr'):
    try:
        col = row.findAll('td')
        team.append(col[2].text)
        squad.append(col[3].text)
        age.append(col[4].text)
        foreigners.append(col[5].text)
        total_market_value.append(col[6].text)
        average_market_value.append(col[7].text)
    except:
        pass

## **Build Dataframe**

In [6]:
df_ekstraklasa = pd.DataFrame({'Team': team[1:],'Squad': squad[1:], 
                              'Age': age[1:], 'Foreigners': foreigners[1:],
                              'Total Value': total_market_value[1:], 
                              'Average value': average_market_value[1:]})

In [7]:
print(df_ekstraklasa)

              Team Squad   Age Foreigners Total Value Average value
0   Legia Warszawa    41  23,4         13  587 tys. €   24,05 mln €
1      Lech Poznań    34  23,7         11  501 tys. €   17,05 mln €
2    Śląsk Wrocław    36  24,7         14  417 tys. €   15,03 mln €
3   Zagłębie Lubin    43  22,9         13  331 tys. €   14,23 mln €
4    Górnik Zabrze    44  24,7          8  302 tys. €   13,30 mln €
5    Piast Gliwice    35  24,8         14  359 tys. €   12,55 mln €
6     Wisła Kraków    38  22,5         10  311 tys. €   11,83 mln €
7      Jagiellonia    43  21,9         11  251 tys. €   10,78 mln €
8    Korona Kielce    42  23,9         10  249 tys. €   10,45 mln €
9    Lechia Gdańsk    39  22,8          7  253 tys. €    9,88 mln €
10  Pogoń Szczecin    34  23,6          7  269 tys. €    9,15 mln €
11     Widzew Łódź    45  22,1         16  179 tys. €    8,08 mln €
12         Zawisza    33  25,2          7  233 tys. €    7,68 mln €
13        Cracovia    34  23,7         10  225 t

## **SEASON 2012-2021**

In [28]:
dct_pl = {}

for m in range(2012,2022):
    dct_pl['df_pl_%s' % m] = pd.DataFrame()

## **List with our needed URLs for each season**

In [9]:
league_urls = 'https://www.transfermarkt.pl/pko-ekstraklasa/startseite/wettbewerb/PL1/plus/?saison_id='

## **Scrapping all data**

In [79]:
for n in range(2012,2022):
        df_soccer1 = None
        url = league_urls + str(n)
        headers = {"User-Agent":"Mozilla/5.0"}
        response = requests.get(url, headers=headers, verify=False)
        soup = BeautifulSoup(response.text, 'html.parser')

        #Table 1 with information about the value
        table = soup.find("table", {"class" : "items"})

        team = []
        squad = []
        age = []
        foreigners = []
        total_market_value = []
        average_market_value = []

        for row in table.findAll('tr'):
            try:
                col = row.findAll('td')
                team.append(col[2].text)
                squad.append(col[3].text)
                age.append(col[4].text)
                foreigners.append(col[5].text)
                total_market_value.append(col[7].text)
                average_market_value.append(col[6].text)
            except:
                pass

        team = [elem.replace('\n','').replace('\xa0','').strip() for elem in team]

        #Table 2 with information about placement, goals and points
        

        df_soccer2 = None

        table2 = soup.findAll("div", {"class" : "responsive-table"})

        team2 = []
        place = []
        matches = []
        difference = []
        pts = []

        if n in range(2013,2020):
          for row in table2[1].findAll('tr'):
                try:
                    col = row.findAll('td')
                    team2.append(col[2].text)
                    place.append(col[0].text)
                    matches.append(col[3].text)
                    difference.append(col[4].text)
                    pts.append(col[5].text)
                except:
                    pass
          for row in table2[2].findAll('tr'):
                try:
                    col = row.findAll('td')
                    team2.append(col[2].text)
                    place.append(8+int(col[0].text))
                    matches.append(col[3].text)
                    difference.append(col[4].text)
                    pts.append(col[5].text)
                except:
                    pass
        else:
            if len(table2) <= 2:
              for row in table2[1].findAll('tr'):
                  try:
                      col = row.findAll('td')
                      team2.append(col[2].text)
                      place.append(col[0].text)
                      matches.append(col[3].text)
                      difference.append(col[4].text)
                      pts.append(col[5].text)
                  except:
                      pass
            else:
              #Sometimes the information you need is in another table
              for row in table2[2].findAll('tr'):
                  try:
                      col = row.findAll('td')
                      team2.append(col[2].text)
                      place.append(col[0].text)
                      matches.append(col[3].text)
                      difference.append(col[4].text)
                      pts.append(col[5].text)
                  except:
                      pass
        
        team2 = [elem.replace('\n','').replace('\xa0','').strip() for elem in team2]

        df_soccer1 = pd.DataFrame({'Team': team[1:], 'Season': n, 'Squad': squad[1:], 'Age': age[1:], 'Foreigners': foreigners[1:],
                                     'Average value': average_market_value[1:], 'Total Value': total_market_value[1:]})

        df_soccer2 = pd.DataFrame({'Team': team2, 'Place': place, 'Matches': matches, 'Difference': difference,
                                 'Points': pts})
        dct_pl['df_pl_%s' % n] = pd.merge(df_soccer1, df_soccer2, how="inner", left_on="Team", right_on="Team")

In [80]:
df_pl_final = pd.concat(dct_pl.values(), ignore_index=True)

In [87]:
#Cleaning strings and remove letters
list_dataframes_conv = (['Average value','Total Value'])

for n in range(0,len(list_dataframes_conv)):
    df_pl_final[list_dataframes_conv[n]] = df_pl_final[list_dataframes_conv[n]]. \
    apply(lambda x: (x + "000").replace(',','') if " tys. €" in x else x).str.replace(' tys. €','')

    df_pl_final[list_dataframes_conv[n]] = df_pl_final[list_dataframes_conv[n]]. \
    apply(lambda x: "0," + x if " Th. €" in x else x).str.replace(' mln €','').str.replace(' mln €','').replace(',',np.NaN)

    df_pl_final[list_dataframes_conv[n]] = df_pl_final[list_dataframes_conv[n]].apply(lambda x: x.replace(',','.'))

df_pl_final['Age'] = df_pl_final['Age'].apply(lambda x: x.replace(',','.'))

In [97]:
#Convert columns for calculation to float
df_pl_final[['Squad','Foreigners','Average value', 'Matches', 'Place','Difference', 'Points']].astype(int)
df_pl_final[['Age','Total value']].astype(float)

ValueError: ignored

In [100]:
df_pl_final.head(100)

,Team,Season,Squad,Age,Foreigners,Average value,Total Value,Place,Matches,Difference,Points
0,Wisła Kraków,2012,32,25.0,13,632000,20.23,8,30,-7,38
1,Śląsk Wrocław,2012,34,24.6,9,594000,20.20,3,30,2,47
2,Legia Warszawa,2012,25,24.2,7,772000,19.30,1,30,37,67
3,Lech Poznań,2012,31,24.2,10,504000,15.63,2,30,24,61
4,Polonia,2012,41,23.6,12,346000,14.20,6,30,11,42
...,...,...,...,...,...,...,...,...,...,...,...
95,Górnik Zabrze,2017,35,22.4,7,91000,3.20,4,37,14,60
96,Legia Warszawa,2018,44,25.0,20,789000,34.73,2,37,17,68
97,Lech Poznań,2018,41,23.6,15,459000,18.80,8,37,1,52
98,Jagiellonia,2018,45,23.8,18,343000,15.43,4,37,3,57
